In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Input, LeakyReLU
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# 加载MNIST数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 数据预处理：归一化
x_train = x_train / 255.0
x_test = x_test / 255.0

x_train = x_train.reshape(-1, 28, 28, 1)  # 保持原始图像形状并添加通道维度
x_test = x_test.reshape(-1, 28, 28, 1)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 数据增强
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.4,
    height_shift_range=0.4,
    zoom_range=0.4,
    shear_range=0.4,
    fill_mode="nearest",
)
datagen.fit(x_train)

# 构建改进后的BP神经网络模型
model = Sequential(
    [
        Input(shape=(28, 28, 1)),  # 使用Input层指定输入形状
        Flatten(),  # 展平图像

        Dense(2048),  # 输入层 -> 隐藏层 1
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(1024),  # 隐藏层 2
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(512),  # 隐藏层 3
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(256),  # 隐藏层 4
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(128),  # 隐藏层 5
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(64),  # 隐藏层 6
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(32),  # 隐藏层 7
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(10, activation="softmax"),  # 输出层
    ]
)

# 编译模型
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),  # 使用RMSprop优化器
    loss="categorical_crossentropy",  # 多分类交叉熵损失
    metrics=["accuracy"],
)

# 回调函数
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=8, min_lr=1e-6, verbose=1
)

early_stopping = EarlyStopping(
    monitor="val_loss", patience=8, restore_best_weights=True
)

# 训练模型
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=128),  # 调整批量大小
    epochs=300,  # 增加训练轮数
    validation_data=(x_test, y_test),
    callbacks=[reduce_lr, early_stopping],
)

# 测试模型
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# 保存模型
model.save("mnist_model_bp_v4.keras")

Epoch 1/300


2025-01-11 15:30:57.861032: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


468/469 [============================>.] - ETA: 0s - loss: 2.4554 - accuracy: 0.1445

2025-01-11 15:31:12.907518: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - 18s 34ms/step - loss: 2.4548 - accuracy: 0.1445 - val_loss: 2.1064 - val_accuracy: 0.2430 - lr: 0.0010
Epoch 2/300
469/469 [==============================] - 15s 33ms/step - loss: 2.1464 - accuracy: 0.1998 - val_loss: 1.9107 - val_accuracy: 0.2560 - lr: 0.0010
Epoch 3/300
469/469 [==============================] - 15s 33ms/step - loss: 2.0569 - accuracy: 0.2335 - val_loss: 1.7987 - val_accuracy: 0.3258 - lr: 0.0010
Epoch 4/300
469/469 [==============================] - 16s 33ms/step - loss: 1.9889 - accuracy: 0.2628 - val_loss: 1.6944 - val_accuracy: 0.3724 - lr: 0.0010
Epoch 5/300
469/469 [==============================] - 16s 34ms/step - loss: 1.9012 - accuracy: 0.3062 - val_loss: 1.4022 - val_accuracy: 0.4915 - lr: 0.0010
Epoch 6/300
469/469 [==============================] - 15s 33ms/step - loss: 1.7946 - accuracy: 0.3596 - val_loss: 1.2415 - val_accuracy: 0.5904 - lr: 0.0010
Epoch 7/300
469/469 [==============================] - 15s 33ms/